In [ ]:
import csv
import math
moviemap={}
moviemap_inv={}
with open('./movies_dataset/ratings.csv','rt')as f:
    data = csv.reader(f)
    i=0
    cnt=0
    for row in data:
        if i!=0:
            x=int(row[1])
            if x in moviemap.keys():
                print("present"+str(x))
            else:
                moviemap[int(row[1])]=cnt
                moviemap_inv[cnt]=int(row[1])
                cnt=cnt+1
        i=i+1
print(cnt)
rows=611
cols=9725
rating_mat=[[0 for i in range(cols)] for j in range(rows)]
test_mat=[[0 for i in range(cols)] for j in range(rows)]
with open('./movies_dataset/ratings.csv','rt')as f:
    data = csv.reader(f)
    i=0
    for row in data:
        if i!=0:
            if i%5==0:
                test_mat[int(row[0])][moviemap[int(row[1])]]=float(row[2])
            else:
                rating_mat[int(row[0])][moviemap[int(row[1])]]=float(row[2])
        i=i+1

In [2]:
#filling missing entries with mean value
import numpy as np
r_m = np.array(rating_mat)
t_m = np.array(test_mat)
for i in range(rows):
    s=0
    cnt=0
    for j in range(cols):
        s += r_m[i][j]
        if r_m[i][j]>0:
            cnt=cnt+1
    if cnt==0:
        print("user"+str(i)+" hasn't rated any movie")
    else:
        s = s/cnt
    for j in range(cols):
        if r_m[i][j]==0:
            r_m[i][j]=s

user0 hasn't rated any movie


In [ ]:
row_to_be_added = np.zeros(9725)
r_m_appended=r_m
for i in range(611,9725):
    r_m_appended = np.vstack((r_m_appended, row_to_be_added))
    if i%100==0:
        print(np.shape(r_m_appended))

In [23]:
print(np.shape(r_m_appended))

(9725, 9725)


In [24]:
np.save('r_m_appended.txt',r_m_appended)

In [4]:
import numpy as np
r_m_appended = np.load('r_m_appended.txt.npy')

In [17]:
np.shape(r_m_appended)

(612, 9725)

In [5]:
w,v = np.linalg.eig(r_m_appended)

In [9]:
eig1 = np.absolute(w)

In [10]:
eig2 = np.sort(eig1)

In [14]:
eig3 = eig2[::-1]

In [15]:
eig3.shape

(9725,)

In [16]:
x = np.mean(eig3)

In [17]:
print(x)

0.3605416621759813


In [18]:
cnt=0
for i in range(cols):
    if eig3[i]>=0.36:
        cnt=cnt+1
print(cnt)

496


In [6]:
k=496

In [21]:
np.save('w',w)

In [3]:
iterations = 10000000
alpha = 1e-7

In [68]:
#gradient descent
u_mat1 = np.random.rand(rows,k)
v_mat1 = np.random.rand(cols,k)
e_mat1 = np.zeros((rows,cols))
for i in range(iterations):
    e_mat1 = r_m - np.dot(u_mat1,np.transpose(v_mat1))
    if i%10==0:
        print("num of iterations: "+str(i))
        print(np.sum(e_mat1)/(rows*cols))
    u_mat1 = u_mat1 + alpha*(np.dot(e_mat1,v_mat1))
    v_mat1 = v_mat1 + alpha*(np.dot(np.transpose(e_mat1),u_mat1))

num of iterations: 0
-120.22777589439825
num of iterations: 10
-33.796812877970005
num of iterations: 20
-10.048182307278873
num of iterations: 30
-3.003138167189613
num of iterations: 40
-0.8970293186013354
num of iterations: 50
-0.26677441485578296
num of iterations: 60
-0.0781352342355448
num of iterations: 70
-0.02167210768506915
num of iterations: 80
-0.004771766828637084
num of iterations: 90
0.0002864903262688113
num of iterations: 100
0.0018001128936994983


KeyboardInterrupt: 

In [65]:
#stochastic gradient descent
u_mat = np.random.rand(rows,k)
v_mat = np.random.rand(cols,k)
e_mat = np.zeros((rows,cols))
u_mat[np.isnan(u_mat)] = np.random.randn(len(u_mat[np.isnan(u_mat)]))
v_mat[np.isnan(v_mat)] = np.random.randn(len(v_mat[np.isnan(v_mat)]))
for _iter in range(iterations):
    for i in range(rows):
        for j in range(cols):
            e_mat[i][j]=r_m[i][j]-np.dot(u_mat[i],np.transpose(v_mat[j]))
    if _iter%5==0:
        print(np.sum(e_mat)/(rows*cols))
        print("_iter: "+str(_iter))
    for i in range(rows):
        for q in range(k):
            u_mat[i][q] = u_mat[i][q]+alpha*np.dot(e_mat[i],np.transpose(np.transpose(v_mat)[q]))
            v_mat[i][q] = v_mat[i][q]+alpha*np.dot(np.transpose(e_mat)[i],np.transpose(np.transpose(u_mat)[q]))
            

-120.3262652433591
_iter: 0
-63.2085786142647
_iter: 5
-33.275000286699694
_iter: 10
-17.527102655435225
_iter: 15
-9.233228490508877
_iter: 20
-4.863737567173378
_iter: 25
-2.561515599899504
_iter: 30
-1.3484679694502202
_iter: 35
-0.7093018792474992
_iter: 40
-0.37251770632720344
_iter: 45
-0.1950617866358719
_iter: 50
-0.10155790363844984
_iter: 55
-0.05228945982590338
_iter: 60
-0.026329270896452418
_iter: 65
-0.012650522562381823
_iter: 70
-0.005443035262957589
_iter: 75
-0.0016453450457470727
_iter: 80
0.0003556759109135029
_iter: 85


KeyboardInterrupt: 

In [66]:
np.save('u_mat',u_mat)#stochastic u

In [67]:
np.save('v_mat',v_mat)#stochastic v

In [69]:
np.save('u_mat1',u_mat1)#batch u

In [70]:
np.save('v_mat1',v_mat1)#batch v

Results:
    Batch: converges after 100 iterations
    stochastic: converges after 80 iterations
    Best alpha: 1e-7

In [81]:
#finding rmse error case a
import math
s=0
cnt=0
for i in range(rows):
    for j in range(cols):
        if test_mat[i][j]>0:
            x = np.dot(u_mat1[i],np.transpose(v_mat1[j]))
            y = test_mat[i][j]-x
            y = y*y
            s = s+y
            cnt=cnt+1
s=s/cnt
s = math.sqrt(s)

In [82]:
print(s)

2.066041585180731


In [83]:
#finding rmse error case b
import math
s=0
cnt=0
for i in range(rows):
    for j in range(cols):
        if test_mat[i][j]>0:
            x = np.dot(u_mat[i],np.transpose(v_mat[j]))
            y = test_mat[i][j]-x
            y = y*y
            s = s+y
            cnt=cnt+1
s=s/cnt
s = math.sqrt(s)
s1=s1/cnt

In [84]:
print(s)

2.066862105341271


Results:
    rmse:
        batch: 2.066041585180731
        stochastic: 2.066862105341271

In [48]:
_lambda = 10

In [49]:
#case1: %5=0 is the test set
r_m2=np.zeros((rows,cols))
for i in range(rows):
    for j in range(cols):
        if j%5!=0:
            r_m2[i][j]=r_m[i][j]
for i in range(rows):
    s=0
    cnt=0
    for j in range(cols):
        s += r_m2[i][j]
        if r_m2[i][j]>0:
            cnt=cnt+1
    if cnt==0:
        print("user"+str(i)+" hasn't rated any movie")
    else:
        s = s/cnt
    for j in range(cols):
        if r_m2[i][j]==0:
            r_m2[i][j]=s

user0 hasn't rated any movie


In [50]:
#case1 
u_mat2 = np.random.rand(rows,k)
v_mat2 = np.random.rand(cols,k)
e_mat2 = np.zeros((rows,cols))
for i in range(iterations):
    e_mat2 = r_m2 - np.dot(u_mat2,np.transpose(v_mat2))
    if i%10==0:
        print("num of iterations: "+str(i))
        print(np.sum(e_mat2)/(0.8*rows*cols))
    u_mat2 = u_mat2*(1-alpha*_lambda) + alpha*(np.dot(e_mat2,v_mat2))
    v_mat2 = v_mat2*(1-alpha*_lambda) + alpha*(np.dot(np.transpose(e_mat2),u_mat2))

num of iterations: 0
-150.40853704538486
num of iterations: 10
-42.26343582558997
num of iterations: 20
-12.561301063537982
num of iterations: 30
-3.753068596792137
num of iterations: 40
-1.120656054573315
num of iterations: 50
-0.33312356774475926
num of iterations: 60
-0.09746985602850065
num of iterations: 70
-0.026950494963246394
num of iterations: 80
-0.005847191576436379
num of iterations: 90
0.00046785182653265463


KeyboardInterrupt: 

In [51]:
#finding the error in case1
import math
cnt=0
s=0
for i in range(rows):
    for j in range(cols):
        if j%5==0:
            x = np.dot(u_mat2[i],np.transpose(v_mat2[j]))
            y = x-r_m[i][j]
            y = y*y
            s=s+y
            cnt=cnt+1
error1 = math.sqrt(s/cnt)

In [52]:
print(error1)

1.8432890179206305


In [53]:
#case2: %5=1 is the test set
r_m2=np.zeros((rows,cols))
for i in range(rows):
    for j in range(cols):
        if j%5!=1:
            r_m2[i][j]=r_m[i][j]
for i in range(rows):
    s=0
    cnt=0
    for j in range(cols):
        s += r_m2[i][j]
        if r_m2[i][j]>0:
            cnt=cnt+1
    if cnt==0:
        print("user"+str(i)+" hasn't rated any movie")
    else:
        s = s/cnt
    for j in range(cols):
        if r_m2[i][j]==0:
            r_m2[i][j]=s

user0 hasn't rated any movie


In [54]:
#case2:
u_mat2 = np.random.rand(rows,k)
v_mat2 = np.random.rand(cols,k)
e_mat2 = np.zeros((rows,cols))
for i in range(iterations):
    e_mat2 = r_m2 - np.dot(u_mat2,np.transpose(v_mat2))
    if i%5==0:
        print("num of iterations: "+str(i))
        print(np.sum(e_mat2)/(0.8*rows*cols))
    u_mat2 = u_mat2*(1-alpha*_lambda) + alpha*(np.dot(e_mat2,v_mat2))
    v_mat2 = v_mat2*(1-alpha*_lambda) + alpha*(np.dot(np.transpose(e_mat2),u_mat2))

num of iterations: 0
-150.672468414522
num of iterations: 5
-78.53471854044322
num of iterations: 10
-42.32354582793858
num of iterations: 15
-23.036011238605163
num of iterations: 20
-12.57715018472425
num of iterations: 25
-6.873536719540991
num of iterations: 30
-3.75728144772142
num of iterations: 35
-2.053527005549217
num of iterations: 40
-1.121787605297491
num of iterations: 45
-0.6121855447515312
num of iterations: 50
-0.33345035269660406
num of iterations: 55
-0.18098704091362672
num of iterations: 60
-0.09759079002394729
num of iterations: 65
-0.05197318857303967
num of iterations: 70
-0.027020283271564003
num of iterations: 75
-0.013370976618673943
num of iterations: 80
-0.005904801320587456
num of iterations: 85
-0.0018208602551369397
num of iterations: 90
0.0004129516930178323


KeyboardInterrupt: 

In [55]:
#finding the error in case2
import math
cnt=0
s=0
for i in range(rows):
    for j in range(cols):
        if j%5==1:
            x = np.dot(u_mat2[i],np.transpose(v_mat2[j]))
            y = x-r_m[i][j]
            y = y*y
            s=s+y
            cnt=cnt+1
error2 = math.sqrt(s/cnt)
print(error2)

1.8417449985970267


In [56]:
#case3: %5=2 is the test set
r_m2=np.zeros((rows,cols))
for i in range(rows):
    for j in range(cols):
        if j%5!=2:
            r_m2[i][j]=r_m[i][j]
for i in range(rows):
    s=0
    cnt=0
    for j in range(cols):
        s += r_m2[i][j]
        if r_m2[i][j]>0:
            cnt=cnt+1
    if cnt==0:
        print("user"+str(i)+" hasn't rated any movie")
    else:
        s = s/cnt
    for j in range(cols):
        if r_m2[i][j]==0:
            r_m2[i][j]=s
u_mat2 = np.random.rand(rows,k)
v_mat2 = np.random.rand(cols,k)
e_mat2 = np.zeros((rows,cols))
for i in range(iterations):
    e_mat2 = r_m2 - np.dot(u_mat2,np.transpose(v_mat2))
    if i%5==0:
        print("num of iterations: "+str(i))
        print(np.sum(e_mat2)/(0.8*rows*cols))
    u_mat2 = u_mat2*(1-alpha*_lambda) + alpha*(np.dot(e_mat2,v_mat2))
    v_mat2 = v_mat2*(1-alpha*_lambda) + alpha*(np.dot(np.transpose(e_mat2),u_mat2))

user0 hasn't rated any movie
num of iterations: 0
-150.4767339689179
num of iterations: 5
-78.47991535167287
num of iterations: 10
-42.31690788356277
num of iterations: 15
-23.04470009814338
num of iterations: 20
-12.58862134180223
num of iterations: 25
-6.883488603735531
num of iterations: 30
-3.764734289160881
num of iterations: 35
-2.058697952230846
num of iterations: 40
-1.1252089686973057
num of iterations: 45
-0.6143752850416307
num of iterations: 50
-0.33481604837366474
num of iterations: 55
-0.18181968116091826
num of iterations: 60
-0.098086798473893
num of iterations: 65
-0.05226040139109074
num of iterations: 70
-0.027179789173023966
num of iterations: 75
-0.013453240943860412
num of iterations: 80
-0.005940772841604251
num of iterations: 85
-0.001829298429712067
num of iterations: 90
0.00042078351352126966


KeyboardInterrupt: 

In [41]:
#finding the error in case3
import math
cnt=0
s=0
for i in range(rows):
    for j in range(cols):
        if j%5==2:
            x = np.dot(u_mat2[i],np.transpose(v_mat2[j]))
            y = x-r_m[i][j]
            y = y*y
            s=s+y
            cnt=cnt+1
error3 = math.sqrt(s/cnt)
print(error3)

1.8437396195499816


In [42]:
#case4: %5=3 is the test set
r_m2=np.zeros((rows,cols))
for i in range(rows):
    for j in range(cols):
        if j%5!=3:
            r_m2[i][j]=r_m[i][j]
for i in range(rows):
    s=0
    cnt=0
    for j in range(cols):
        s += r_m2[i][j]
        if r_m2[i][j]>0:
            cnt=cnt+1
    if cnt==0:
        print("user"+str(i)+" hasn't rated any movie")
    else:
        s = s/cnt
    for j in range(cols):
        if r_m2[i][j]==0:
            r_m2[i][j]=s
u_mat2 = np.random.rand(rows,k)
v_mat2 = np.random.rand(cols,k)
e_mat2 = np.zeros((rows,cols))
for i in range(iterations):
    e_mat2 = r_m2 - np.dot(u_mat2,np.transpose(v_mat2))
    if i%5==0:
        print("num of iterations: "+str(i))
        print(np.sum(e_mat2)/(0.8*rows*cols))
    u_mat2 = u_mat2*(1-alpha*_lambda) + alpha*(np.dot(e_mat2,v_mat2))
    v_mat2 = v_mat2*(1-alpha*_lambda) + alpha*(np.dot(np.transpose(e_mat2),u_mat2))

user0 hasn't rated any movie
num of iterations: 0
-150.2225510139485
num of iterations: 5
-78.35414854844424
num of iterations: 10
-42.248213697698226
num of iterations: 15
-23.006052007894187
num of iterations: 20
-12.566684628432121
num of iterations: 25
-6.87101220463418
num of iterations: 30
-3.7576445200983364
num of iterations: 35
-2.0546813230785577
num of iterations: 40
-1.1229467563562525
num of iterations: 45
-0.613114808387636
num of iterations: 50
-0.33412750777314226
num of iterations: 55
-0.18145762564277645
num of iterations: 60
-0.09791106552548369
num of iterations: 65
-0.052190966416676046
num of iterations: 70
-0.027170973846921954
num of iterations: 75
-0.013478979508210755
num of iterations: 80
-0.005986196105463268
num of iterations: 85
-0.0018859273307016279
num of iterations: 90
0.0003577822455270243


KeyboardInterrupt: 

In [43]:
#finding the error in case4
import math
cnt=0
s=0
for i in range(rows):
    for j in range(cols):
        if j%5==3:
            x = np.dot(u_mat2[i],np.transpose(v_mat2[j]))
            y = x-r_m[i][j]
            y = y*y
            s=s+y
            cnt=cnt+1
error4 = math.sqrt(s/cnt)
print(error4)

1.8429115284967217


In [44]:
#case5: %5=4 is the test set
r_m2=np.zeros((rows,cols))
for i in range(rows):
    for j in range(cols):
        if j%5!=4:
            r_m2[i][j]=r_m[i][j]
for i in range(rows):
    s=0
    cnt=0
    for j in range(cols):
        s += r_m2[i][j]
        if r_m2[i][j]>0:
            cnt=cnt+1
    if cnt==0:
        print("user"+str(i)+" hasn't rated any movie")
    else:
        s = s/cnt
    for j in range(cols):
        if r_m2[i][j]==0:
            r_m2[i][j]=s
u_mat2 = np.random.rand(rows,k)
v_mat2 = np.random.rand(cols,k)
e_mat2 = np.zeros((rows,cols))
for i in range(iterations):
    e_mat2 = r_m2 - np.dot(u_mat2,np.transpose(v_mat2))
    if i%5==0:
        print("num of iterations: "+str(i))
        print(np.sum(e_mat2)/(0.8*rows*cols))
    u_mat2 = u_mat2*(1-alpha*_lambda) + alpha*(np.dot(e_mat2,v_mat2))
    v_mat2 = v_mat2*(1-alpha*_lambda) + alpha*(np.dot(np.transpose(e_mat2),u_mat2))

user0 hasn't rated any movie
num of iterations: 0
-150.4776994760011
num of iterations: 5
-78.47057503859085
num of iterations: 10
-42.30622685828795
num of iterations: 15
-23.035666084858587
num of iterations: 20
-12.58188618603692
num of iterations: 25
-6.878816453560874
num of iterations: 30
-3.7616475224932047
num of iterations: 35
-2.056735480930283
num of iterations: 40
-1.124005592626266
num of iterations: 45
-0.61366775780447
num of iterations: 50
-0.3344248095091857
num of iterations: 55
-0.18162661597691476
num of iterations: 60
-0.09801617858677723
num of iterations: 65
-0.052264592325839694
num of iterations: 70
-0.027229267322353586
num of iterations: 75
-0.0135299218386738
num of iterations: 80
-0.0060336848448056665
num of iterations: 85
-0.0019318381334653336
num of iterations: 90
0.0003125637073174237


KeyboardInterrupt: 

In [45]:
#finding the error in case5
import math
cnt=0
s=0
for i in range(rows):
    for j in range(cols):
        if j%5==4:
            x = np.dot(u_mat2[i],np.transpose(v_mat2[j]))
            y = x-r_m[i][j]
            y = y*y
            s=s+y
            cnt=cnt+1
error5 = math.sqrt(s/cnt)
print(error5)

1.8427013714066556


In [46]:
avg_error = (error1+error2+error3+error4+error5)/5

In [47]:
print(avg_error)

1.8426032931588812
